<a href="https://colab.research.google.com/github/cchen744/uhi-extreme-heat-response/blob/main/notebooks/01_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I will quantify Urban Heat Isalnd Effect and Extreme Heat within one single city (Chicago). This data preprocessing workflow will be applited to more cities later.
Notably, here are following definitions of variables of interest:
- **Urban Heat Island**: Based on land surface temperture, it is defined as SUHI(day) = LST_urb(day) − LST_rur(day), where LST_urb(day) = Aggregate urban pixels (mean or median, predefined) and LST_rur(day) = Aggregate rural reference pixels.
- **Extreme Heat Window**: three consecutive days when the daily mean landsurface temperature overpasses 95 percentile of its historical data.
- **Urban & Rural Definition**：

  - Urban: US Census Urbanized Area
  - Rural: Spatial mean of all non-urban, non-water pixels within the same UA



In [79]:
# Core
import ee
import geemap
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
ee.Authenticate()
ee.Initialize(project='extremeweatheruhi')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Global Parameters

In [70]:
# Import US Census urbanized area
ua = ee.FeatureCollection("projects/extremeweatheruhi/assets/uac20_2025")

# Find Chicago first
name_list=ua.aggregate_array("NAME20").getInfo()
print([name for name in name_list if name and "Chicago" in name])

# Got this from above codes
CITY_NAME = 'Chicago, IL--IN'

START_DATE = "2013-06-01"
END_DATE   = "2019-08-31"

# LST choice = "Night"
LST_BAND = "LST_Night_1km"
QC_BAND  = "QC_Night"

# Aggregation choice
AGG_FUNC = "mean"

# Extreme heat definition (LST-based)
EXTREME_PERCENTILE = 95
EXTREME_CONSEC_DAYS = 3

# Pixel count threshold (drop day if too sparse)
MIN_URBAN_PIXELS = 30
MIN_RURAL_PIXELS = 30

# rural ring definition (meters)
RING_OUTER_M = 30000   # 30 km
RING_INNER_M = 5000    # 5 km

# aggregation scales
LST_SCALE_M = 1000     # MODIS LST nominal 1km
LCZ_SCALE_M = 100      # LCZ map nominal 100m

['Chicago, IL--IN']


## 1. Load Urbanized Area

In [71]:
city = ua.filter(ee.Filter.stringContains("NAME20", "Chicago, IL"))
city_geom = city.geometry()

print("city features:", city.size().getInfo())
print("city area km2:", city_geom.area().divide(1e6).getInfo())

# Sanity check - visualize the Chicago urbanized area
# Define style parameters
vis_params_filled = {
    'color': '9370DB',
    'width': 1,
    'fillColor': '00000044'
}
Map = geemap.Map()
Map.centerObject(city, 8)
Map.addLayer(city, vis_params_filled, "Chicago UA")
Map

city features: 1
city area km2: 6147.151289346154


Map(center=[41.82877621700015, -87.90672170347597], controls=(WidgetControl(options=['position', 'transparent_…

Land cover - urban/rural mask

### LCZ Classification Reference
https://developers.google.com/earth-engine/datasets/catalog/RUB_RUBCLIM_LCZ_global_lcz_map_latest

| GEE Code | LCZ Class | Name & Description | Thermal Feature |
| :--- | :--- | :--- | :--- |
| **1** | LCZ 1 | **Compact high-rise** (CBD) | High heat storage, slow cooling. |
| **2** | LCZ 2 | **Compact mid-rise** | High density, limited ventilation. |
| **3** | LCZ 3 | **Compact low-rise** | Dense housing, hot roofs. |
| **4** | LCZ 4 | **Open high-rise** | Better ventilation. |
| **5** | LCZ 5 | **Open mid-rise** | Common campus/residential. |
| **6** | LCZ 6 | **Open low-rise** (Suburbs) | Cooler due to gardens/lawns. |
| **7** | LCZ 7 | **Lightweight low-rise** | Poor insulation. |
| **8** | LCZ 8 | **Large low-rise** (Mall/Warehouse) | **Hottest** (Big roof, no shadow). |
| **9** | LCZ 9 | **Sparsely built** | Natural setting. |
| **10** | LCZ 10 | **Heavy industry** | Waste heat emissions. |
| **11** | LCZ A | **Dense trees** | **Coolest** (Shading). |
| **12** | LCZ B | **Scattered trees** | Cooler than paved areas. |
| **13** | LCZ C | **Bush, scrub** | Moderate cooling. |
| **14** | LCZ D | **Low plants** (Grass/Farm) | Cools quickly at night. |
| **15** | LCZ E | **Bare rock or paved** | Dry and hot. |
| **16** | LCZ F | **Bare soil or sand** | Heats up very fast. |
| **17** | LCZ G | **Water** | Cooling by day, warms at night. |


In [73]:
# LCZ global map (100m)
lcz_img = ee.ImageCollection("RUB/RUBCLIM/LCZ/global_lcz_map/latest").first()
lcz = lcz_img.select("LCZ_Filter")

BUILT_MIN, BUILT_MAX = 1, 10
WATER_CODE = 17

is_built = lcz.gte(BUILT_MIN).And(lcz.lte(BUILT_MAX))
is_water = lcz.eq(WATER_CODE)
is_natural = is_built.Not().And(is_water.Not())

# Urban region: UA polygon (geometry)
urban_region = city_geom

# Rural region: ring outside UA
outer = city_geom.buffer(RING_OUTER_M)
inner = city_geom.buffer(RING_INNER_M)
rural_region = outer.difference(inner)

# Masks
# Urban pixels: inside UA, exclude water
urban_mask = is_water.Not().clip(urban_region)

# Rural pixels: inside ring, only natural (exclude built + water)
rural_mask = is_natural.clip(rural_region)

# Visualize
Map = geemap.Map()
Map.centerObject(city_geom, 8)
Map.addLayer(city, {}, "UA")
Map.addLayer(rural_region, {"color": "yellow"}, "Rural ring (vector)")
Map.addLayer(urban_mask.selfMask(), {"palette": ['#ffaaaa']}, "Urban mask (UA - water)") # Light red for urban
Map.addLayer(rural_mask.selfMask(), {"palette": ['#aaffaa']}, "Rural mask (ring natural)") # Light green for rural
Map

Map(center=[41.82877621700015, -87.90672170347597], controls=(WidgetControl(options=['position', 'transparent_…

In [77]:
hist = lcz.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=city_geom,
    scale=LCZ_SCALE_M,
    maxPixels=1e13
).getInfo()

print(hist)

{'LCZ_Filter': {'1': 1167.4627450980392, '10': 966.7647058823527, '11': 46316.1450980393, '12': 83067.16862745097, '14': 86476.03137254954, '15': 856.7882352941176, '16': 1822.8941176470585, '17': 10629.521568627433, '2': 1846, '3': 10789, '4': 253.1921568627451, '5': 26776.549019607843, '6': 442319.1058823514, '8': 90983.58431372543, '9': 22528.71764705881}}


## 2. Load Daily MODIS LST

In [75]:
lst_raw = (
    ee.ImageCollection("MODIS/061/MYD11A1")
      .filterBounds(urban_region)
      .filterDate(START_DATE, END_DATE)
      .select([LST_BAND, QC_BAND])
)
print("lst_raw size:", lst_raw.size().getInfo())

lst_raw size: 2282


In [76]:
def clean_lst(img):
    lst = img.select(LST_BAND)
    qc  = img.select(QC_BAND).toUint16()

    # bits 0-1: Mandatory QA
    mandatory = qc.bitwiseAnd(3)

    # keep produced pixels: 0 or 1
    produced = mandatory.lte(1)

    # drop fill (0)
    valid = lst.neq(0)

    good = produced.And(valid)

    # scale: 0.02 K -> C
    lst_c = lst.multiply(0.02).subtract(273.15)

    return (
        lst_c.updateMask(good)
             .rename(LST_BAND)
             .copyProperties(img, ["system:time_start"])
    )

lst_clean = lst_raw.map(clean_lst)

In [63]:
good_date = "2013-06-02"

# Get the cleaned image directly from the lst_clean ImageCollection
# The lst_clean collection already has the clean_lst function applied to all images.
# Filter lst_clean for the specific date
img_good_clean_collection = lst_clean.filterDate(good_date, ee.Date(good_date).advance(1, "day"))

if img_good_clean_collection.size().getInfo() == 0:
    print(f"No cleaned image found for {good_date}. Setting count to 0.")
    cnt = {'LST_Night_1km': 0}
else:
    img_good_clean = ee.Image(img_good_clean_collection.first())

    cnt = img_good_clean.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=city_geom,
        scale=1000,
        maxPixels=1e13
    ).getInfo()

print("count after QA:", cnt)

count after QA: {'LST_Night_1km': 1270}


Data spatial aggregation (UA x Day)

In [ ]:
def make_daily_table(start, end):
    ic = (ee.ImageCollection("MODIS/061/MYD11A1")
          .filterBounds(urban_region)   # bounds 用 urban_region 足够（rural 在附近）
          .filterDate(start, end)
          .select([LST_BAND, QC_BAND])
          .map(clean_lst))

    reducer = ee.Reducer.mean() if AGG_FUNC == "mean" else ee.Reducer.median()

    def agg(img):
        date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")

        # Apply masks
        urb = img.updateMask(urban_mask)
        rur = img.updateMask(rural_mask)

        urb_stat = urb.reduceRegion(
            reducer=reducer,
            geometry=urban_region,
            scale=LST_SCALE_M,
            maxPixels=1e13
        )
        rur_stat = rur.reduceRegion(
            reducer=reducer,
            geometry=rural_region,
            scale=LST_SCALE_M,
            maxPixels=1e13
        )

        urb_n = urb.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=urban_region,
            scale=LST_SCALE_M,
            maxPixels=1e13
        )
        rur_n = rur.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=rural_region,
            scale=LST_SCALE_M,
            maxPixels=1e13
        )

        return ee.Feature(None, {
            "date": date,
            "LST_urb": urb_stat.get(LST_BAND),
            "LST_rur": rur_stat.get(LST_BAND),
            "urban_n": urb_n.get(LST_BAND),
            "rural_n": rur_n.get(LST_BAND),
        })

    return ic.map(agg)

fc = make_daily_table('2013-06-01', '2013-06-30')  # FeatureCollection
print("fc size:", fc.size().getInfo())

In [91]:
fc = ee.FeatureCollection(fc)
df = geemap.ee_to_df(fc)
df.head()

,LST_rur,date,rural_n,urban_n,LST_urb
0,8.354013,2013-06-01,11,0,NaN
1,14.899787,2013-06-02,2321,1217,15.614281
2,5.054137,2013-06-03,2600,4615,5.282349
3,8.129132,2013-06-04,3914,1109,8.859029
4,9.744976,2013-06-05,674,442,9.907543


In [ ]:
def month_starts(start_date, end_date):
    s = datetime.strptime(start_date, "%Y-%m-%d").replace(day=1)
    e = datetime.strptime(end_date, "%Y-%m-%d")
    cur = s
    while cur < e:
        nxt = cur + relativedelta(months=1)
        yield cur.strftime("%Y-%m-%d"), nxt.strftime("%Y-%m-%d")
        cur = nxt

dfs = []
for s, e in month_starts(START_DATE, END_DATE):
    fc = make_daily_table(s, e)
    fc = ee.FeatureCollection(fc)
    df = geemap.ee_to_df(fc)

    df = df.dropna()
    df = df[(df["urban_n"] >= MIN_URBAN_PIXELS) & (df["rural_n"] >= MIN_RURAL_PIXELS)]

    if len(df) > 0:
        dfs.append(df)

    print(s, "->", e, "rows:", len(df))

df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
df_all.head(), len(df_all)

2013-06-01 -> 2013-07-01 rows: 22
2013-07-01 -> 2013-08-01 rows: 25
2013-08-01 -> 2013-09-01 rows: 29
2013-09-01 -> 2013-10-01 rows: 26
2013-10-01 -> 2013-11-01 rows: 23


2013-11-01 -> 2013-12-01 rows: 23
2013-12-01 -> 2014-01-01 rows: 20
2014-01-01 -> 2014-02-01 rows: 24
2014-02-01 -> 2014-03-01 rows: 25
2014-03-01 -> 2014-04-01 rows: 27
2014-04-01 -> 2014-05-01 rows: 20
2014-05-01 -> 2014-06-01 rows: 22
2014-06-01 -> 2014-07-01 rows: 17
2014-07-01 -> 2014-08-01 rows: 29
2014-08-01 -> 2014-09-01 rows: 27
2014-09-01 -> 2014-10-01 rows: 26
2014-10-01 -> 2014-11-01 rows: 24
2014-11-01 -> 2014-12-01 rows: 21
2014-12-01 -> 2015-01-01 rows: 23
2015-01-01 -> 2015-02-01 rows: 26
2015-02-01 -> 2015-03-01 rows: 21
2015-03-01 -> 2015-04-01 rows: 23
2015-04-01 -> 2015-05-01 rows: 25
2015-05-01 -> 2015-06-01 rows: 20
2015-06-01 -> 2015-07-01 rows: 19
2015-07-01 -> 2015-08-01 rows: 25
2015-08-01 -> 2015-09-01 rows: 28
2015-09-01 -> 2015-10-01 rows: 24
2015-10-01 -> 2015-11-01 rows: 24
2015-11-01 -> 2015-12-01 rows: 24
2015-12-01 -> 2016-01-01 rows: 23
2016-01-01 -> 2016-02-01 rows: 25
2016-02-01 -> 2016-03-01 rows: 23
2016-03-01 -> 2016-04-01 rows: 19
2016-04-01 -> 

2016-07-01 -> 2016-08-01 rows: 25
2016-08-01 -> 2016-09-01 rows: 20
2016-09-01 -> 2016-10-01 rows: 22
2016-10-01 -> 2016-11-01 rows: 26
2016-11-01 -> 2016-12-01 rows: 23
2016-12-01 -> 2017-01-01 rows: 20
2017-01-01 -> 2017-02-01 rows: 21


Export Daily LST

In [ ]:
if not df_all.empty:
    df_all["date"] = pd.to_datetime(df_all["date"])
    for c in ["LST_urb", "LST_rur", "urban_n", "rural_n"]:
        df_all[c] = pd.to_numeric(df_all[c], errors="coerce")

    df_all = df_all.dropna().sort_values("date").drop_duplicates(subset=["date"]).reset_index(drop=True)
    df_all["SUHI"] = df_all["LST_urb"] - df_all["LST_rur"]

    # Extreme heat label (LST-based, using urban series)
    thr = np.percentile(df_all["LST_urb"], EXTREME_PERCENTILE)
    df_all["hot_day"] = (df_all["LST_urb"] >= thr).astype(int)
    df_all["hot_run"] = df_all["hot_day"].rolling(EXTREME_CONSEC_DAYS).sum()
    df_all["is_extreme"] = (df_all["hot_run"] >= EXTREME_CONSEC_DAYS).astype(int)

out_cols = ["date","SUHI","LST_urb","LST_rur","urban_n","rural_n","is_extreme"]
df_out = df_all[out_cols].copy()

df_out.to_csv("chicago_daily_SUHI_urban_vs_ringrural.csv", index=False)
print("saved rows:", len(df_out))
df_out.head()